In [23]:
import pandas as pd
import datetime

In [24]:
CLUBS = ['SSG', '키움','LG','KT','KIA', '롯데', '삼성', '두산', '한화', 'NC']
HOLIDAYS = []
HOLIDAYS.append(datetime.date(2024, 4, 10))
HOLIDAYS.append(datetime.date(2024, 5, 5))
HOLIDAYS.append(datetime.date(2024, 5, 18))
HOLIDAYS.append(datetime.date(2024, 6, 6))
HOLIDAYS.append(datetime.date(2024, 8, 15))

print(len(CLUBS))
print(len(HOLIDAYS))

10
5


## Matches, Calendar, Initial Plan

아래와 같은 형태로 필요

```
  "initialPlan": [
    {
      "index": 0,
      "datetime": "2023-04-01 14:00:00",
      "home": "SSG",
      "away": "KIA",
      "consecutive": 2,
      "weekend": 1,
      "holiday": 2
    }, ...
   ],
  "calendar": [
    {
      "index": 0,
      "datetime": "2023-04-01 14:00:00",
      "consecutive": 2,
      "weekend": 1,
      "holiday": 2
    }, ...
  ],
  "matches": [
    {
      "index": 0,
      "home": "SSG",
      "away": "KIA",
      "consecutive": 2
    }, ...
  ]

```

In [25]:


initial_plan_arr = []
calendar_arr = []
matches_arr = []

input_sheet = pd.read_excel('./baseball_2024.xlsx', engine='openpyxl', sheet_name="Input")
match_index = 0

matche_object = {}

consecutive =0
day_of_week = 0
holiday_cnt = 0

initial_plan_arr = []
calendar_arr = []
match_arr = []

calendar_index =0


total_match_cnt = 0

for row_index, row in input_sheet.iterrows():
    
    if row['요일'] in ['월'] :
        continue
    
    # date 만들기
    year = 2024
    month = row['월']
    day = row['일']
    
    if consecutive == 0:
        match_start_date = datetime.date(year, month, day) # 3연전이 시작하는 날짜

    consecutive += 1
    
    
    if row['요일'] in ['토', '일'] :
        day_of_week =1
        holiday_cnt +=1
    elif datetime.date(year, month, day) in HOLIDAYS: # 현재 일자가 휴일에 포함되는지...
        holiday_cnt +=1  
    
    
    for stadium in ['잠실', '고척', '문학', '수원', '대전', '대구', '광주', '사직', '창원']:
        if not pd.isna(row[stadium])   :
            away_team = str(row[stadium]).split('-')[0]
            home_team = str(row[stadium]).split('-')[1]
            if stadium not in matche_object:
                matche_object[stadium] = {}
                matche_object[stadium]['stadium'] = stadium
                matche_object[stadium]['index'] = match_index
                matche_object[stadium]['datetime'] = match_start_date
                matche_object[stadium]['home'] = home_team
                matche_object[stadium]['away'] = away_team
        
        
        
   
    
    if row['요일'] in ['목', '일'] : # 항상 목요일 또는 일요일에 끝난다.
        
        calendar_object = {}
        calendar_object['index'] = calendar_index
        calendar_object['datetime'] = str(match_datetime) + " 00:00:00"
        calendar_object['consecutive'] = consecutive
        calendar_object['weekend'] = day_of_week
        calendar_object['holiday'] = holiday_cnt
        calendar_arr.append(calendar_object)
        calendar_index +=1
        
        for match_object_value in matche_object.values():
            stadium = match_object_value['stadium']
            index = match_object_value['index']
            match_datetime = match_object_value['datetime']
            home = match_object_value['home']
            away = match_object_value['away']
            
            initail_plan_object = {}
            initail_plan_object["index"] = match_index
            initail_plan_object["datetime"] = str(match_datetime) + " 00:00:00"
            
            initail_plan_object["home"] = home
            initail_plan_object["away"] = away
            initail_plan_object["consecutive"] = consecutive
            initail_plan_object["weekend"] = day_of_week
            initail_plan_object["holiday"] = holiday_cnt
            initial_plan_arr.append(initail_plan_object)
            
          
            
            match_object = {}
            match_object['index'] = match_index
            match_object['home'] = home
            match_object['away'] = away
            match_object['consecutive'] = consecutive
            match_arr.append(match_object)
        
            total_match_cnt +=  consecutive
            match_index +=1
            
            
        matche_object = {}
        consecutive = 0
        day_of_week = 0
        holiday_cnt = 0
        
        
for row in initial_plan_arr:
    print(row)
    
print('----')

print(match_index, total_match_cnt)

{'index': 0, 'datetime': '2024-03-23', 'home': 'LG', 'away': '한화', 'consecutive': 2, 'weekend': 1, 'holiday': 2}
{'index': 1, 'datetime': '2024-03-23', 'home': 'SSG', 'away': '롯데', 'consecutive': 2, 'weekend': 1, 'holiday': 2}
{'index': 2, 'datetime': '2024-03-23', 'home': 'KT', 'away': '삼성', 'consecutive': 2, 'weekend': 1, 'holiday': 2}
{'index': 3, 'datetime': '2024-03-23', 'home': 'KIA', 'away': '키움', 'consecutive': 2, 'weekend': 1, 'holiday': 2}
{'index': 4, 'datetime': '2024-03-23', 'home': 'NC', 'away': '두산', 'consecutive': 2, 'weekend': 1, 'holiday': 2}
{'index': 5, 'datetime': '2024-03-26', 'home': 'LG', 'away': '삼성', 'consecutive': 3, 'weekend': 0, 'holiday': 0}
{'index': 6, 'datetime': '2024-03-26', 'home': 'SSG', 'away': '한화', 'consecutive': 3, 'weekend': 0, 'holiday': 0}
{'index': 7, 'datetime': '2024-03-26', 'home': 'KT', 'away': '두산', 'consecutive': 3, 'weekend': 0, 'holiday': 0}
{'index': 8, 'datetime': '2024-03-26', 'home': 'KIA', 'away': '롯데', 'consecutive': 3, 'weeken

## Basic Info
### teams, distance matrix

In [26]:
import json
with open ('./basic_info.json', 'r', encoding='utf8') as file:
    basic_info = json.load(file)

In [27]:
engine_input = {}
engine_input['initialPlan'] = initial_plan_arr
engine_input['calendar'] = calendar_arr
engine_input['matches'] = match_arr
engine_input['teams'] = basic_info['teams']
engine_input['distanceMatrix'] = basic_info['distanceMatrix']


In [28]:
engine_input['initialPlan'] 

[{'index': 0,
  'datetime': '2024-03-23',
  'home': 'LG',
  'away': '한화',
  'consecutive': 2,
  'weekend': 1,
  'holiday': 2},
 {'index': 1,
  'datetime': '2024-03-23',
  'home': 'SSG',
  'away': '롯데',
  'consecutive': 2,
  'weekend': 1,
  'holiday': 2},
 {'index': 2,
  'datetime': '2024-03-23',
  'home': 'KT',
  'away': '삼성',
  'consecutive': 2,
  'weekend': 1,
  'holiday': 2},
 {'index': 3,
  'datetime': '2024-03-23',
  'home': 'KIA',
  'away': '키움',
  'consecutive': 2,
  'weekend': 1,
  'holiday': 2},
 {'index': 4,
  'datetime': '2024-03-23',
  'home': 'NC',
  'away': '두산',
  'consecutive': 2,
  'weekend': 1,
  'holiday': 2},
 {'index': 5,
  'datetime': '2024-03-26',
  'home': 'LG',
  'away': '삼성',
  'consecutive': 3,
  'weekend': 0,
  'holiday': 0},
 {'index': 6,
  'datetime': '2024-03-26',
  'home': 'SSG',
  'away': '한화',
  'consecutive': 3,
  'weekend': 0,
  'holiday': 0},
 {'index': 7,
  'datetime': '2024-03-26',
  'home': 'KT',
  'away': '두산',
  'consecutive': 3,
  'weekend': 0

In [29]:
import json
js = json.dumps(engine_input, indent='\t', ensure_ascii=False)

fp =  open ('input_2024.json', 'w', encoding='utf-8')
fp.write(js)
fp.close()